In [176]:
import pandas as pd

In [177]:
data_path = '/home/achara/data/achara/'

In [178]:
ws_17 = pd.read_csv(data_path + '2017_JP_WX.csv')

In [179]:
ws_17.head()

,DateTime,Temperature_Air_HC2,BarometricPressure_Air_HC2,DielectricWavePeriod_Soil_CS616,RelativeHumidity_Air_HC2,Moisture_Soil_CS616,Temperature_Soil_CS616,SolarRad_Air_LiCor,WindDirection_Air_RMY85006,WindDirection-StdDev_Air_RMY85006,...,Temperature_Air_WXT,BarometricPressure_Air_WXT,RelativeHumidity_Air_WXT,WindDirection_Air_WXT,WindDirection-StdDev_Air_WXT,WindSpeed_Air_WXT,Precipitation-Hail_Air_WXT,WindSpeed-Max_Air_WXT,Precipitation-RainDuration_Air_WXT,WindSpeed-Min_Air_WXT
0,1/1/2017 0:00,-0.356037,997.518519,21.896225,94.253704,0.131365,1.528815,0.0,226.748500,47.694074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/1/2017 1:00,-0.203712,997.000000,21.899866,92.513559,0.131454,1.528544,0.0,234.364407,40.453780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1/1/2017 2:00,-0.175179,997.000000,21.900850,92.244643,0.131478,1.533117,0.0,207.388054,38.515893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1/1/2017 3:00,-0.211704,997.000000,21.901289,91.811111,0.131488,1.542955,0.0,164.736296,25.343704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/1/2017 4:00,-0.321241,996.462963,21.904211,94.668519,0.131560,1.546150,0.0,222.037037,24.615593,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observation: There are several `0` values for `SolarRad_Air_LiCor` sensor data.

## Extracting Solar Radiation data

In [180]:
ws_17_solar = ws_17[['DateTime', 'SolarRad_Air_LiCor']]
ws_17_solar.head()

,DateTime,SolarRad_Air_LiCor
0,1/1/2017 0:00,0.0
1,1/1/2017 1:00,0.0
2,1/1/2017 2:00,0.0
3,1/1/2017 3:00,0.0
4,1/1/2017 4:00,0.0


In [181]:
ws_17_solar.SolarRad_Air_LiCor.describe()

count    26514.000000
mean       166.113749
std        263.170066
min          0.000000
25%          0.018971
50%          8.526046
75%        245.840250
max       1353.333333
Name: SolarRad_Air_LiCor, dtype: float64

In [182]:
len(ws_17_solar[ws_17_solar.SolarRad_Air_LiCor == 0])

4613

In [183]:
# Checking for NA values
len(ws_17_solar[ws_17_solar.SolarRad_Air_LiCor.isna()])

10092

In [184]:
ws_17_solar.DateTime.head()

0    1/1/2017 0:00
1    1/1/2017 1:00
2    1/1/2017 2:00
3    1/1/2017 3:00
4    1/1/2017 4:00
Name: DateTime, dtype: object

In [185]:
def remove_time(datetime: str) -> str:
    return datetime.split(' ')[0]

In [186]:
ws_17_solar.DateTime = ws_17_solar.DateTime.map(remove_time)

/home/achara/achara_env/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [187]:
ws_17_solar.DateTime.head()

0    1/1/2017
1    1/1/2017
2    1/1/2017
3    1/1/2017
4    1/1/2017
Name: DateTime, dtype: object

## Find mean value of `SolarRad_Air_LiCor` by Date

In [188]:
ws_17_solar_mean = ws_17_solar.groupby('DateTime')[['SolarRad_Air_LiCor']].mean()

In [189]:
ws_17_solar_mean.head()

,SolarRad_Air_LiCor
DateTime,
1/1/2017,73.880391
1/1/2018,0.027000
1/10/2017,38.902959
1/11/2017,84.137819
1/12/2017,12.829765


In [190]:
len(ws_17_solar[ws_17_solar.SolarRad_Air_LiCor.isna()])

10092

## Need to replace the `NA` values by the mean of `SolarRad_Air_LiCor` when matched with `DateTime`

In [ ]:
ws_17_solar['SolarRad_Air_LiCor'] = ws_17_solar['SolarRad_Air_LiCor'].fillna(
    ws_17_solar.DateTime.map(lambda date: ws_17_solar_mean.at[str(date), 'SolarRad_Air_LiCor']))

In [ ]:
'''
There still can be DateTime values for which all SolarRad_Air_LiCor values were NA because of which NA values will still persist for these DateTime values.
I will replace these NA values with the overall mean.
'''

ws_17_solar.SolarRad_Air_LiCor.fillna(ws_17_solar.SolarRad_Air_LiCor.mean(), inplace=True)